In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
f2t=pd.read_csv('Face2Text.csv') 

In [3]:
f2t.filename.nunique()

4076

In [4]:
f2t['description']=f2t['description'].str.replace("[^a-zA-Z]"," ")
f2t['description']=f2t['description'].str.lower()

In [5]:
for i in range(len(f2t['description'])):
    f2t['description'][i]='seqstart '+f2t['description'][i]+' seqend'

C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
vocab = pd.read_csv('vocab.csv',header=None)

In [7]:
len(vocab)

1336

In [8]:
wordtoin={}
intoword={}
ind=0
for i in vocab[0]:
    wordtoin[i]=ind
    intoword[ind]=i
    ind+=1 

In [9]:
wordtoin

{'whch': 0,
 'th': 1,
 'loosely': 2,
 'shaggy': 3,
 'deep': 4,
 'shaped': 5,
 'naturally': 6,
 'it': 7,
 'aged': 8,
 'emphasising': 9,
 'had': 10,
 'pinned': 11,
 'imitating': 12,
 'stubbly': 13,
 'unable': 14,
 'frame': 15,
 'mouthed': 16,
 'decay': 17,
 'blank': 18,
 'surprise': 19,
 'between': 20,
 'plaid': 21,
 'match': 22,
 'styled': 23,
 'highlighted': 24,
 'enck': 25,
 'reddish': 26,
 'kinky': 27,
 'acting': 28,
 'small': 29,
 'suntanned': 30,
 'distance': 31,
 'built': 32,
 'bony': 33,
 'type': 34,
 'pastel': 35,
 'combed': 36,
 'oink': 37,
 'stud': 38,
 'tint': 39,
 'ibig': 40,
 'teetj': 41,
 'amek': 42,
 'round': 43,
 'ann': 44,
 'eyerbows': 45,
 'believe': 46,
 'narrow': 47,
 'wearling': 48,
 'matching': 49,
 'french': 50,
 'blusher': 51,
 'mediterranean': 52,
 'awe': 53,
 'top': 54,
 'believing': 55,
 'upstyle': 56,
 'seeon': 57,
 'cover': 58,
 'wrinkly': 59,
 'am': 60,
 'veiny': 61,
 'straw': 62,
 'under': 63,
 'toothbrush': 64,
 'dense': 65,
 'tattoos': 66,
 'otherwise': 

In [10]:
maxv=0
for i in f2t['description']:
    maxv=max(maxv,len(i.split()))
print(maxv)

77


In [11]:
from keras.applications.xception import Xception
model = Xception(weights='imagenet')

In [12]:
from keras.models import Model
model_new = Model(model.input, model.layers[-2].output)

In [13]:
from keras.applications.xception import preprocess_input
def preprocess(imag):
    x = imag
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [14]:
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [15]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for i in range(len(descriptions)):
            n+=1
            # retrieve the photo feature
            photo = photos[f2t['filename'][i]]
            desc=descriptions[i]
                # encode the sequence
            seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
            for i in range(1, len(seq)):
                    # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=maxv)[0]
                    # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=len(vocab))[0]
                    # store
                X1.append(photo)
                X2.append(in_seq)
                y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [75]:
EMBEDDING_FILE=os.path.join('glove.6B.200d.txt')

In [76]:
import numpy as np
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE,encoding='utf8'))

In [16]:
vecdim = 200

In [78]:
embedding_matrix = np.zeros((len(vocab), vecdim))

for word, i in wordtoin.items():
    embedding_vector=None
    if(word in embeddings_index.keys()):
        embedding_vector = embeddings_index[word]
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [79]:
embedding_matrix.shape

(1336, 200)

In [17]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.xception import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(maxv,))
se1 = Embedding(len(vocab), vecdim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(len(vocab), activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [81]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
epochs = 10
number_pics_per_bath = 3
steps = len(f2t['description'])//number_pics_per_bath

In [19]:
model.load_weights('model_x_19.h5')

In [20]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(maxv):
        sequence = [wordtoin[w] for w in in_text.split() if w in wordtoin]
        sequence = pad_sequences([sequence], maxlen=maxv)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = intoword[yhat]
        in_text += ' ' + word
        if word == 'seqend':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [21]:
import dlib
from imutils import face_utils

shape_predictor = dlib.shape_predictor(r'shape_predictor_5_face_landmarks.dat')
fa = face_utils.facealigner.FaceAligner(shape_predictor, desiredFaceWidth=299, desiredLeftEye=(0.4, 0.4))

In [ ]:
import cv2
import numpy as np
import onnx
import onnxruntime as ort
from onnx_tf.backend import prepare

def area_of(left_top, right_bottom):
    """
    Compute the areas of rectangles given two corners.
    Args:
        left_top (N, 2): left top corner.
        right_bottom (N, 2): right bottom corner.
    Returns:
        area (N): return the area.
    """
    hw = np.clip(right_bottom - left_top, 0.0, None)
    return hw[..., 0] * hw[..., 1]

def iou_of(boxes0, boxes1, eps=1e-5):
    """
    Return intersection-over-union (Jaccard index) of boxes.
    Args:
        boxes0 (N, 4): ground truth boxes.
        boxes1 (N or 1, 4): predicted boxes.
        eps: a small number to avoid 0 as denominator.
    Returns:
        iou (N): IoU values.
    """
    overlap_left_top = np.maximum(boxes0[..., :2], boxes1[..., :2])
    overlap_right_bottom = np.minimum(boxes0[..., 2:], boxes1[..., 2:])

    overlap_area = area_of(overlap_left_top, overlap_right_bottom)
    area0 = area_of(boxes0[..., :2], boxes0[..., 2:])
    area1 = area_of(boxes1[..., :2], boxes1[..., 2:])
    return overlap_area / (area0 + area1 - overlap_area + eps)

def hard_nms(box_scores, iou_threshold, top_k=-1, candidate_size=200):
    """
    Perform hard non-maximum-supression to filter out boxes with iou greater
    than threshold
    Args:
        box_scores (N, 5): boxes in corner-form and probabilities.
        iou_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
        candidate_size: only consider the candidates with the highest scores.
    Returns:
        picked: a list of indexes of the kept boxes
    """
    scores = box_scores[:, -1]
    boxes = box_scores[:, :-1]
    picked = []
    indexes = np.argsort(scores)
    indexes = indexes[-candidate_size:]
    while len(indexes) > 0:
        current = indexes[-1]
        picked.append(current)
        if 0 < top_k == len(picked) or len(indexes) == 1:
            break
        current_box = boxes[current, :]
        indexes = indexes[:-1]
        rest_boxes = boxes[indexes, :]
        iou = iou_of(
            rest_boxes,
            np.expand_dims(current_box, axis=0),
        )
        indexes = indexes[iou <= iou_threshold]

    return box_scores[picked, :]

def predict(width, height, confidences, boxes, prob_threshold, iou_threshold=0.5, top_k=-1):
    """
    Select boxes that contain human faces
    Args:
        width: original image width
        height: original image height
        confidences (N, 2): confidence array
        boxes (N, 4): boxes array in corner-form
        iou_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
    Returns:
        boxes (k, 4): an array of boxes kept
        labels (k): an array of labels for each boxes kept
        probs (k): an array of probabilities for each boxes being in corresponding labels
    """
    boxes = boxes[0]
    confidences = confidences[0]
    picked_box_probs = []
    picked_labels = []
    for class_index in range(1, confidences.shape[1]):
        probs = confidences[:, class_index]
        mask = probs > prob_threshold
        probs = probs[mask]
        if probs.shape[0] == 0:
            continue
        subset_boxes = boxes[mask, :]
        box_probs = np.concatenate([subset_boxes, probs.reshape(-1, 1)], axis=1)
        box_probs = hard_nms(box_probs,
           iou_threshold=iou_threshold,
           top_k=top_k,
           )
        picked_box_probs.append(box_probs)
        picked_labels.extend([class_index] * box_probs.shape[0])
    if not picked_box_probs:
        return np.array([]), np.array([]), np.array([])
    picked_box_probs = np.concatenate(picked_box_probs)
    picked_box_probs[:, 0] *= width
    picked_box_probs[:, 1] *= height
    picked_box_probs[:, 2] *= width
    picked_box_probs[:, 3] *= height
    return picked_box_probs[:, :4].astype(np.int32), np.array(picked_labels), picked_box_probs[:, 4]

video_capture = cv2.VideoCapture(0)

onnx_path = r'C:\Users\shrey\Desktop\Face2Text\ultra_light_640.onnx'
onnx_model = onnx.load(onnx_path)
predictor = prepare(onnx_model)
ort_session = ort.InferenceSession(onnx_path)
input_name = ort_session.get_inputs()[0].name

frame_count=0
while True:
    ret, frame = video_capture.read()
    if frame is not None:
        h, w, _ = frame.shape

        # preprocess img acquired
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # convert bgr to rgb
        img = cv2.resize(img, (640, 480)) # resize
        img_mean = np.array([127, 127, 127])
        img = (img - img_mean) / 128
        img = np.transpose(img, [2, 0, 1])
        img = np.expand_dims(img, axis=0)
        img = img.astype(np.float32)

        confidences, boxes = ort_session.run(None, {input_name: img})
        boxes, labels, probs = predict(w, h, confidences, boxes, 0.7)
        areas={}
        for i in range(boxes.shape[0]):
            box = boxes[i, :]
            x1, y1, x2, y2 = box
            area=np.abs(x1-x2)*np.abs(y1-y2)
            areas[(x1,y1,x2,y2)]=(area)
        areas={k: v for k, v in sorted(areas.items(), key=lambda item: item[1],reverse=True)}
        boxes = list(areas.keys())[:1]
            
        
        colorList=[(0,0,255),(255,0,0),(0,0,255)]
        for i in range(len(boxes)):
            box = boxes[i]
            x1, y1, x2, y2 = box
            cv2.rectangle(frame, (x1, y1), (x2, y2), colorList[i%3], 2)
            #cv2.rectangle(frame, (x1, y2 - 20), (x2, y2), (80,18,236), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            
            #x1, y1, x2, y2 = boxes[0,:]
            # convert to gray scale image
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # align and resize
            aligned_face = fa.align(frame, gray, dlib.rectangle(left = x1, top=y1, right=x2, bottom=y2))
            aligned_face = cv2.resize(aligned_face, (299,299))  #299*299*3
            
            aligned_face = encode(aligned_face).reshape((1,2048))

            desc = greedySearch(aligned_face)
            #print(len(desc))
            cv2.putText(frame, desc[:70], (0, (0+10)), font, 0.5, colorList[i%3], 1)
            cv2.putText(frame, desc[70:140], (0, ( 10+ 15)), font, 0.5, colorList[i%3], 1)
            cv2.putText(frame, desc[140:210], (0, ( 30+10)), font, 0.5, colorList[i%3], 1)
            cv2.putText(frame, desc[210:280], (0, ( 45+10)), font, 0.5, colorList[i%3], 1)
            cv2.putText(frame, desc[280:350], (0, (60+10)), font, 0.5, colorList[i%3], 1)
            cv2.putText(frame, desc[350:420], (0, (75+10)), font, 0.5, colorList[i%3], 1)
            cv2.putText(frame, desc[420:], (0, (90+10)), font, 0.5, colorList[i%3], 1)

        cv2.imshow('Video', frame)
        #frame_count += 1
        # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\PythonCPU\lib\site-packages\onnx_tf\common\__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\PythonCPU\lib\site-packages\onnx_tf\common\handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\PythonCPU\lib\site-packages\onnx_tf\common\handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\PythonCPU\lib\site-packages\onnx_tf\common\handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\PythonCPU\lib\site-packages\onnx_tf\common\handler_helper.py:34: UserWarning: Fail to get since_version of GatherND in domain `` with max_inclusive_version=9. Set to 1.
  ha

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
